In [44]:
from math import fabs
from ortools.linear_solver import pywraplp
import random

solver = pywraplp.Solver.CreateSolver('GLOP')
while(True):
    constraints_num = 2

    x1 = solver.NumVar(0,50,'x1')# variable creating
    x2 = solver.NumVar(0,50,'x2')

    b1 = random.randrange(2,32,1)
    ct1 = solver.Constraint(0,b1,'ct')
    a1 = random.randrange(1,7,1);
    a2 = random.randrange(-7,7,1);
    ct1.SetCoefficient(x1,a1)
    ct1.SetCoefficient(x2,a2)
    b2 = random.randrange(2,32,1)
    ct2 = solver.Constraint(0,b2,'ct')
    a3 = random.randrange(1,7,1)
    ct2.SetCoefficient(x1,a3)
    if a2 < 0:
        a4 = random.randrange(1,7,1)
    else:
        a4 = random.randrange(-7,7,1)
    ct2.SetCoefficient(x2,a4)

    c1 = random.randrange(1,7,1)
    c2 = random.randrange(1,7,1)
    obj = solver.Objective()
    obj.SetCoefficient(x1, c1)
    obj.SetCoefficient(x2, c2)
    obj.SetMaximization()
    solver.Solve()
    xx1 = x1.solution_value()
    xx2 = x2.solution_value()
    if xx1 != 0 and xx2 != 0 and fabs(round(xx1)-xx1) < 0.00000001 and fabs(round(xx2)-xx2) < 0.00000001 :
        print(str(c1)+"x1 + "+str(c2)+"x2 -> Max")
        print(str(a1)+"x1 + "+str(a2)+"x2 <= "+ str(b1))
        print(str(a3)+"x1 + "+str(a4)+"x2 <= "+ str(b2))
        print('Solution:')
        print('Objective value =', round(xx1+xx2))
        print('x1 =', round(xx1))
        print('x2 =', round(xx2))
        print("\n")



1x1 +3x2 -> Max
3x1 + -2x2 <= 9
6x1 + 2x2 <= 18
Solution:
Objective value = 5
x1 = 2
x2 = 3


6x1 +5x2 -> Max
2x1 + 0x2 <= 30
6x1 + -2x2 <= 24
Solution:
Objective value = 60
x1 = 15
x2 = 45


1x1 +3x2 -> Max
2x1 + 6x2 <= 18
4x1 + -6x2 <= 9
Solution:
Objective value = 5
x1 = 3
x2 = 2


2x1 +3x2 -> Max
4x1 + -4x2 <= 27
3x1 + 2x2 <= 30
Solution:
Objective value = 12
x1 = 6
x2 = 6


4x1 +6x2 -> Max
4x1 + -4x2 <= 7
6x1 + 2x2 <= 16
Solution:
Objective value = 4
x1 = 2
x2 = 2


1x1 +2x2 -> Max
4x1 + 0x2 <= 24
2x1 + -4x2 <= 19
Solution:
Objective value = 9
x1 = 6
x2 = 3


5x1 +5x2 -> Max
5x1 + -4x2 <= 7
2x1 + 1x2 <= 26
Solution:
Objective value = 18
x1 = 8
x2 = 10


3x1 +3x2 -> Max
1x1 + 3x2 <= 18
4x1 + 2x2 <= 12
Solution:
Objective value = 6
x1 = 0
x2 = 6


3x1 +4x2 -> Max
4x1 + 5x2 <= 26
2x1 + -4x2 <= 9
Solution:
Objective value = 6
x1 = 4
x2 = 2


5x1 +1x2 -> Max
4x1 + -4x2 <= 16
1x1 + 3x2 <= 24
Solution:
Objective value = 14
x1 = 9
x2 = 5


6x1 +3x2 -> Max
3x1 + -1x2 <= 31
6x1 + 2x2 <= 24


KeyboardInterrupt: 